In [1]:
import pyodbc
import csv
import xml.etree.cElementTree as et
import numpy as np
import pandas as pd
import re

In [40]:
EMOTIONS = ['joy', 'anger', 'sadness', 'fear']

## Fetching ISEAR Dataset

In [41]:
MDB = "./ISEAR_0/isear_databank/isear_databank.mdb"
DRV = "{Microsoft Access Driver (*.mdb, *.accdb)}"
PWD = "pw"

In [42]:
conn = pyodbc.connect('DRIVER={}; DBQ={}; PWD={}'.format(DRV, MDB, PWD))
cursor = conn.cursor()

In [43]:
QUERY = 'SELECT * FROM data;' 
rows = cursor.execute(QUERY).fetchall()
ISEAR_df = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])
cursor.close()
conn.close()
ISEAR_df

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,SELF,RELA,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE
0,11001.0,1.0,1.0,1.0,1.0,33.0,1.0,2.0,6.0,1.0,...,3.0,3.0,2.0,0.0,joy,4,3,0110011,"During the period of falling in love, each tim...",1
1,11001.0,1.0,1.0,1.0,1.0,33.0,1.0,2.0,6.0,1.0,...,2.0,2.0,0.0,0.0,fear,3,2,0110012,When I was involved in a traffic accident.,1
2,11001.0,1.0,1.0,1.0,1.0,33.0,1.0,2.0,6.0,1.0,...,2.0,1.0,0.0,0.0,anger,1,3,0110013,When I was driving home after several days of...,1
3,11001.0,1.0,1.0,1.0,1.0,33.0,1.0,2.0,6.0,1.0,...,1.0,1.0,0.0,2.0,sadness,4,4,0110014,When I lost the person who meant the most to me.,1
4,11001.0,1.0,1.0,1.0,1.0,33.0,1.0,2.0,6.0,1.0,...,0.0,2.0,0.0,0.0,disgust,4,4,0110015,The time I knocked a deer down - the sight of ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7661,331062.0,1.0,33.0,62.0,2.0,21.0,2.0,1.0,7.0,7.0,...,2.0,2.0,3.0,0.0,anger,1,2,3310623,Two years back someone invited me to be the tu...,1
7662,331062.0,1.0,33.0,62.0,2.0,21.0,2.0,1.0,7.0,7.0,...,2.0,0.0,1.0,1.0,sadness,4,3,3310624,I had taken the responsibility to do something...,1
7663,331062.0,1.0,33.0,62.0,2.0,21.0,2.0,1.0,7.0,7.0,...,2.0,2.0,0.0,0.0,disgust,1,2,3310625,I was at home and I heard a loud sound of spit...,1
7664,331062.0,1.0,33.0,62.0,2.0,21.0,2.0,1.0,7.0,7.0,...,2.0,0.0,2.0,0.0,shame,1,3,3310626,I did not do the homework that the teacher had...,1


In [44]:
ISEAR_df = ISEAR_df.iloc[:, [36, 40]]
ISEAR_df.columns = ['ISEAR_labels', 'ISEAR_sentences']
ISEAR_df

,ISEAR_labels,ISEAR_sentences
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,disgust,The time I knocked a deer down - the sight of ...
...,...,...
7661,anger,Two years back someone invited me to be the tu...
7662,sadness,I had taken the responsibility to do something...
7663,disgust,I was at home and I heard a loud sound of spit...
7664,shame,I did not do the homework that the teacher had...


In [45]:
ISEAR_df['ISEAR_labels'].value_counts()

ISEAR_labels
anger      1096
sadness    1096
disgust    1096
shame      1096
fear       1095
joy        1094
guilt      1093
Name: count, dtype: int64

In [46]:
ISEAR_df_filtered = ISEAR_df[ISEAR_df.ISEAR_labels.isin(EMOTIONS)].reset_index(drop=True)
ISEAR_df_filtered

,ISEAR_labels,ISEAR_sentences
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,joy,When I got a letter offering me the Summer job...
...,...,...
4376,sadness,This issue worried me rather than saddening me...
4377,joy,I received a letter from a distant friend.
4378,fear,My parents were out and I was the eldest at ho...
4379,anger,Two years back someone invited me to be the tu...


In [11]:
ISEAR_CSV = "./datasets_csv/ISEAR_dataset.csv"
ISEAR_df_filtered.to_csv(ISEAR_CSV)

## Fetching AffectiveText Dataset

In [ ]:
XML1 = "./AffectiveText.Semeval.2007/AffectiveText.test/affectivetext_test.xml"
XML2 = "./AffectiveText.Semeval.2007/AffectiveText.trial/affectivetext_trial.xml"
LABELS1 = "./AffectiveText.Semeval.2007/AffectiveText.test/affectivetext_test.emotions.gold"
LABELS2 = "./AffectiveText.Semeval.2007/AffectiveText.trial/affectivetext_trial.emotions.gold"
VALENCE1 = "./AffectiveText.Semeval.2007/AffectiveText.test/affectivetext_test.valence.gold"
VALENCE2 = "./AffectiveText.Semeval.2007/AffectiveText.trial/affectivetext_trial.valence.gold"

In [ ]:
tree=et.parse(XML2)
root=tree.getroot()

In [ ]:
AFFECTIVETEXT_sentences = []
for instance in root.iter('instance'):
    AFFECTIVETEXT_sentences.append(instance.text)

AFFECTIVETEXT_sentences

['Mortar assault leaves at least 18 dead',
 'Goal delight for Sheva',
 'Nigeria hostage feared dead is freed',
 'Bombers kill shoppers',
 'Vegetables, not fruit, slow brain decline',
 'PM: Havana deal a good experiment',
 'Kate is marrying Doherty',
 'NASA revisiting life on Mars question',
 'Happy birthday, iPod',
 'Alonso would be happy to retire with three titles',
 "Madonna's New Tot 'Happy at Home' in London",
 "Nicole Kidman asks dad to help stop husband's drinking",
 'United Finds Good Connection in Win',
 "'Runway': Making Good Without Making Nice",
 "We were 'arrogant and stupid' over Iraq, says US diplomat",
 'Bad reasons to be good',
 "Madonna's new baby's daddy didn't realize adoption was `for good'",
 "Women in their 60s 'are perfectly good mothers'",
 "We're a pretty kind 'bully'",
 "Moderate drinking reduces men's heart attack risk",
 'Tom Cruise and Katie Holmes set wedding date',
 'Kidnapped AP photographer freed in Gaza',
 'Bush Insists Troops Stay in Iraq, Predicts M